До самого пайплайна нужно загрузить необходимые файлы и создать директории - папку для работы (AIMsetfinder) и внутри нее еще одну папку tmp для хранения временных файлов (!!!! - важно - потому что часть файлов сохраняется туда и потом используется)

In [ ]:
!mkdir AIMsetfinder
!cd AIMsetfinder
!mkdir tmp
!cd ..

Скачиваем необходимые файлы

In [ ]:
!wget https://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/integrated_call_samples_v3.20130502.ALL.panel

--2024-06-27 09:58:02--  https://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/integrated_call_samples_v3.20130502.ALL.panel
Resolving ftp.1000genomes.ebi.ac.uk (ftp.1000genomes.ebi.ac.uk)... 193.62.193.167
Connecting to ftp.1000genomes.ebi.ac.uk (ftp.1000genomes.ebi.ac.uk)|193.62.193.167|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 55156 (54K)
Saving to: ‘integrated_call_samples_v3.20130502.ALL.panel’

integrated_call_sam 100%[===================>]  53.86K   116KB/s    in 0.5s    

2024-06-27 09:58:04 (116 KB/s) - ‘integrated_call_samples_v3.20130502.ALL.panel’ saved [55156/55156]



In [ ]:
!wget -nH --cut-dirs=4 -r -p -e robots=off -A 'ALL.*' https://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/

--2024-06-27 09:58:07--  https://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/
Resolving ftp.1000genomes.ebi.ac.uk (ftp.1000genomes.ebi.ac.uk)... 193.62.193.167
Connecting to ftp.1000genomes.ebi.ac.uk (ftp.1000genomes.ebi.ac.uk)|193.62.193.167|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘index.html.tmp’

index.html.tmp          [  <=>               ]  19.90K  87.0KB/s    in 0.2s    

2024-06-27 09:58:08 (87.0 KB/s) - ‘index.html.tmp’ saved [20382]

Removing index.html.tmp since it should be rejected.

--2024-06-27 09:58:08--  https://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/?C=N;O=D
Reusing existing connection to ftp.1000genomes.ebi.ac.uk:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘index.html?C=N;O=D.tmp’

index.html?C=N;O=D.     [ <=>                ]  19.90K  --.-KB/s    in 0.03s   

2024-06-27 09:58:08 (582 KB/s) - ‘index.html?C=N;O=D.tmp’ saved [203

Практическая работа. Работа с библиотекой.

In [ ]:
import hail as hl
hl.init()
import pandas as pd
import numpy as np
import os
import re

Prepare - удаляем группу AMR (смешанные американцы) и создаем тестовую и обучающую выборки

In [ ]:
!grep -E 'AFR|EUR|SAS|EAS' annotation.txt > annotation_noAMR.txt

grep: annotation.txt: No such file or directory


STEP0 - отбираем только би-аллельные SNP и убирает инделы

In [ ]:
chr1 = hl.import_vcf('ALL.chr1.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz', force_bgz=True)

Initializing Hail with default parameters...
Running on Apache Spark version 3.5.1
SparkUI available at http://a298ff5653ee:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.131-11d9b2ff89da
LOGGING: writing to /content/hail-20240627-1002-0.2.131-11d9b2ff89da.log


In [ ]:
#only bi-allelic SNP
chr1 = chr1.filter_rows(hl.len(chr1.alleles) == 2)
#remove indels
chr1 = chr1.filter_rows(hl.is_snp(chr1.alleles[0], chr1.alleles[1]))

In [ ]:
#Получаем тестовую и обучающую выборку
#Обозначение переменных
testindsperdeme = 100
demes = 5
filenameInds = "integrated_call_samples_v3.20130502.ALL.panel"
filenameIndsTest = "annotationTest.txt"
filenameIndsTraining = "annotationTraining.txt"


size = [testindsperdeme for i in range(demes)]
a = pd.read_csv(filenameInds, sep='\t' )
pops = np.sort(np.unique(a.iloc[:, 2]))
inds = a.iloc[:, [0, 2]]
noinds = np.zeros(len(pops), dtype=int)
testset, trainingset = [], []

for i in range(len(pops)):
    indslocal = inds[inds.iloc[:, 1] == pops[i]].iloc[:, 0]
    noinds[i] = np.sum(inds.iloc[:, 1] == pops[i])
    s = np.random.choice(noinds[i], size[i], replace=False)
    testset.extend(indslocal.iloc[s])
    trainingset.extend(indslocal.iloc[np.setdiff1d(np.arange(noinds[i]), s)])

#Создаем пустые файлы с заголовком
os.system(f" head -n 1 {filenameInds} > {filenameIndsTest}")
os.system(f" head -n 1 {filenameInds} > {filenameIndsTraining}")

#Создаем файлы с аннотацией образцов для обучающего и тестового образца по отобранным именам
for test_id in testset:
    os.system(f"grep -w {test_id} {filenameInds} >> {filenameIndsTest}")

for train_id in trainingset:
    os.system(f"grep -w {train_id} {filenameInds} >> {filenameIndsTraining}")

STEP1 - считаем частоты SNP

In [ ]:
table = hl.import_table("integrated_call_samples_v3.20130502.ALL.panel").key_by("sample")
mt = chr1

# объединяем данные из аннотации с данными из vcf файла
mt = mt.annotate_cols(pheno=table[mt.s])

#Список популяций в файле
pops = list(table.aggregate(hl.agg.collect_as_set(table.super_pop)))

# Инициализация таблицы первой популяцией
mt_pop = mt.filter_cols(mt.pheno.super_pop == pops[0])
mt_pop = hl.variant_qc(mt_pop)
mt_pop = mt_pop.annotate_rows(alt = mt_pop.variant_qc.AF[1])
freq_table = mt_pop.rows().select('alt').rename({"alt": f'{pops[0]}'})

# Цикл по оставшимся популяциям
for pop in pops[1:]:
  mt_pop = mt.filter_cols(mt.pheno.super_pop == pop)
  mt_pop = hl.variant_qc(mt_pop)
  mt_pop = mt_pop.annotate_rows(alt = mt_pop.variant_qc.AF[1])
  pop_freq = mt_pop.rows().select('alt').rename({"alt": f'{pop}'})

  # Объединяем таблицы
  freq_table = freq_table.join(pop_freq)

2024-06-27 10:11:52.757 Hail: INFO: Reading table without type imputation
  Loading field 'sample' as type str (not specified)
  Loading field 'pop' as type str (not specified)
  Loading field 'super_pop' as type str (not specified)
  Loading field 'gender' as type str (not specified)


In [ ]:
freq_table.show()

2024-06-27 10:12:47.550 Hail: INFO: scanning VCF for sortedness...
2024-06-27 10:15:54.635 Hail: INFO: Coerced prefix-sorted VCF, requiring additional sorting within data partitions on each query.


,,,,,,
locus,alleles,SAS,AFR,AMR,EAS,EUR
locus<GRCh37>,array<str>,float64,float64,float64,float64,float64
1:10505,"[""A"",""T""]",0.00e+00,7.56e-04,0.00e+00,0.00e+00,0.00e+00
1:10506,"[""C"",""G""]",0.00e+00,7.56e-04,0.00e+00,0.00e+00,0.00e+00
1:10511,"[""G"",""A""]",0.00e+00,0.00e+00,1.44e-03,0.00e+00,0.00e+00
1:10539,"[""C"",""A""]",1.02e-03,0.00e+00,1.44e-03,0.00e+00,9.94e-04
1:10542,"[""C"",""T""]",0.00e+00,0.00e+00,0.00e+00,9.92e-04,0.00e+00
1:10579,"[""C"",""A""]",0.00e+00,7.56e-04,0.00e+00,0.00e+00,0.00e+00
1:10642,"[""G"",""A""]",0.00e+00,1.29e-02,1.44e-03,2.98e-03,0.00e+00
1:11008,"[""C"",""G""]",7.16e-02,1.35e-01,9.65e-02,3.67e-02,8.85e-02


STEP2 - фильтрация SNP по информативности

In [ ]:
#Задаем уровень информативности, при котором мы оставляем SNP
informativenessBound = 0.9
#Вводим дополнительные функции для подсчета информативности
def xlogx2(x):
    z = xlogx(x) + xlogx(1 - x)
    return z

def xlogx(x):
    z = hl.if_else(x == 0, 0, x * hl.log(x))
    return z
#Считаем количесвто строк в файле
nss = freq_table.count()
#Считаем средние
freq_table = freq_table.annotate(
    mean_row=(freq_table.AFR + freq_table.SAS + freq_table.EUR + freq_table.AMR + freq_table.EAS) / len(pops)
)
freq_table = freq_table.annotate(
    mean_row_log=(xlogx2(freq_table.AFR) + xlogx2(freq_table.SAS) + xlogx2(freq_table.EUR) + xlogx2(freq_table.AMR) + xlogx2(freq_table.EAS)) / len(pops)
)
#Считаем информативность
freq_table = freq_table.annotate(info= -xlogx2(freq_table.mean_row) + freq_table.mean_row_log)
#Фильтруем таблицу по информативности (в порядке убывания) и отбираем 10% самых информативных SNP
informativenessQuantile = freq_table.order_by(hl.desc(freq_table.info)).head(hl.eval(hl.int32(hl.floor((1 - informativenessBound)* nss))))
#Создаем новый vcf-файл с отобранными SNP по информативности
informativenessQuantile = informativenessQuantile.key_by("locus")
mt = chr1.filter_rows(hl.is_defined(informativenessQuantile[chr1.locus]))
#Сохраняем файл
hl.export_vcf(mt, 'tmp/filter_chr1.vcf.bgz')

STEP3 - делим имеющийся файл на маленькие

In [ ]:
mt = hl.import_vcf('tmp/filter_chr1.vcf.bgz')
stepsize = 10000
# Вычисляем количество сегментов и преобразуем в список
segments = hl.range(0, hl.int32((mt.count_rows() - 1) // stepsize + 1)).collect()
# Добавляем столбец с номером строки
mt = mt.annotate_rows(row_index=hl.scan.count())

# Фильтруем строки по номеру

# Цикл по сегментам
for j in segments[0]:
    start = j * stepsize
    end = hl.eval(min((j + 1) * stepsize, mt.count_rows()))
    # Фильтруем строки по номеру
    segment_mt = mt.filter_rows((mt.row.row_index >= start) & (mt.row.row_index < end))
    hl.export_vcf(segment_mt, f"tmp/chr_segment{j}.vcf.bgz")

STEP4 - считаем кол-во строк в новом vcf файле

In [ ]:
nss = hl.eval(mt.count()[0])

STEP5 - переводим vcf.gz файлы в hail matrix table для более быстрого чтения

In [ ]:
def genotype_to_number(gt):
    z = hl.case().when(gt.is_het(), 1).when(gt.is_hom_ref(), 0).when(gt.is_hom_var(), 2).or_missing()
    return z
#Создаем список имен файлов для шага 6
filenames = []
for j in segments[0]:
    mt = hl.import_vcf(f"tmp/chr_segment{j}.vcf.bgz")
    mt = mt.annotate_entries(GT=genotype_to_number(mt.GT))
    mt.write(f'tmp/chr_segment{j}.mt', overwrite=True)
    filenames.append(f'tmp/chr_segment{j}')

STEP6 - получаем список AIMs по обучающей выборке

Функции для 6го шага

In [ ]:
def find_aims_hail(filenames_mt, table, num_aim, sampleAIMs):
    """
    Находит AIMs (Ancestry Informative Markers) из списка Hail Matrix Table.

    Args:
        filenames_mt: Список путей к файлам Hail Matrix Table.
        table: Hail Table с информацией о популяциях.
        num_aim: Количество AIMs для поиска.
        sampleAIMs: файл, куда будут складываться AIMs

    Returns:
        Hail Matrix Table с выбранными AIMs.
    """

    while len(sampleAIMs) < num_aim:
        best = []
        all_errors = []
        for filename in filenames_mt:
            best_snp, error = find_next_AIM(filename, sampleAIMs, table)
            best.append(best_snp)
            all_errors.append(error)
        #Находим AIM  с минимальной ошибкой
        best_aim = best[all_errors.index(min(all_errors))]
        #Добавляем лучший SNP к таблице AIMs
        sampleAIMs.append(best_aim)
        best.remove(best_aim )
        all_errors.remove(min(all_errors))

    return sampleAIMs

def find_next_AIM(filename, sampleAIMs, table):
    samples_in_deme = table.aggregate(hl.agg.counter(table.super_pop))
    no = hl.if_else(len(sampleAIMs) == 0, 0, len(sampleAIMs))
    sample = hl.read_matrix_table(f'{filename}.mt')
    sample_vcf= hl.import_vcf(f'{filename}.vcf.bgz')
    freqs_training = getfreqs(sample_vcf, table)

    error = []
    for i in range(sample.count_rows()):
        snp_num = i
        e = App_test_error(sample, freqs_training, sample_names, snp_num, table)
        error.append(e)

    min_error = min(error)
    best_snp = error.index(min_error) #находим snp с минимальной ошибкой
    #ищем название snp
    locus_list = sample_vcf.locus.collect()
    sample_next_AIM = locus_list[best_snp]
    #return sample_next_AIM, min_error
    return sample_next_AIM, min_error

def getfreqs(mt, table_annotation): # mt - .vcf.gz файлы из шага 5, table_annotation - это таблица с описанием образцов
    locus_list = mt.locus.collect()
    #объединяем данные из аннотации с данными из vcf файла
    mt = mt.annotate_cols(pheno=table_annotation[mt.s])

    #Список популяций в файле
    pops = list(table_annotation.aggregate(hl.agg.collect_as_set(table_annotation.super_pop)))

    # Инициализация таблицы первой популяцией
    mt_pop = mt.filter_cols(mt.pheno.super_pop == pops[0])
    mt_pop = hl.variant_qc(mt_pop)
    mt_pop = mt_pop.annotate_rows(alt = mt_pop.variant_qc.AC[1]).key_rows_by("locus")
    freq_list = mt_pop.alt.collect()
    locus_freq_table = hl.Table.parallelize(
        [{"locus": locus, "frequency": freq} for locus, freq in zip(locus_list, freq_list)],
        hl.tstruct(locus=hl.tlocus("GRCh37"), frequency=hl.tint32),
    ).rename({"frequency": f'{pops[0]}'})
    table = locus_freq_table.key_by("locus")
    # Цикл по оставшимся популяциям
    for pop in pops[1:]:
        mt_pop = mt.filter_cols(mt.pheno.super_pop == pop)
        mt_pop = hl.variant_qc(mt_pop)
        mt_pop = mt_pop.annotate_rows(alt = mt_pop.variant_qc.AC[1]).key_rows_by("locus")
        freq_list = mt_pop.alt.collect()
        locus_freq_table = hl.Table.parallelize(
            [{"locus": locus, "frequency": freq} for locus, freq in zip(locus_list, freq_list)],
            hl.tstruct(locus=hl.tlocus("GRCh37"), frequency=hl.tint32)).rename({"frequency": f'{pop}'}).key_by("locus")
        table = table.join(locus_freq_table, how="left")
    return table

def log_prediction_naiveBayes(sample_test, table_with_freq, sample_names, number_of_snp):
  res = np.zeros((len(sample_names), 5))
  for j in range(len(sample_names)):
    selected_mt = sample_test.filter_cols(sample_test.s == f'{sample_names[j]}').GT.collect()[number_of_snp]
    for k in range(5):
      counts = [int(re.search(r'\d+(\.\d+)?', str(i)).group(0)) for i in table_with_freq.select(f"{pops[k]}").collect()][number_of_snp]
      res[j, :][k] += np.log(1 + counts) * selected_mt + np.log((1 + diploid) * samples_in_deme[pops[k]] - counts + 1) * (1 + diploid - selected_mt)
  else:
    return res

def prediction_naive_bayes(sample_test, counts, sample_names, number_of_snp):
  res = log_prediction_naiveBayes(sample_test, counts, sample_names, number_of_snp)
  # Вычисляем экспоненту для каждого значения в res, вычитая максимальное значение
  res = np.exp(res - np.max(res))
  # Нормализуем каждую строку так, чтобы сумма элементов в строке была равна 1
  res = res / res.sum(axis=1, keepdims=True)
  return res

def logloss_error(prediction_table):
  """
  Вычисляет logloss error для Hail Table с вероятностями.

  Args:
      prediction_table (Hail Table): Таблица, содержащая столбцы с вероятностями
          принадлежности к популяциям и столбец 'super_pop' с истинными классами.

  Returns:
      float: Значение logloss error.
  """

  # Получаем список названий популяций из колонок
  populations = ['EAS', 'SAS', 'AFR', 'EUR', 'AMR']

  # Создаем список выражений для получения вероятности для истинного класса
  probs_for_correct_classes_expr = [
    hl.if_else(prediction_table.super_pop == population,
               prediction_table[population],
               hl.literal(0))
    for population in populations
  ]

  # Вычисляем среднее логарифма вероятности для истинного класса
  logloss = - prediction_table.aggregate(hl.agg.mean(hl.log(hl.sum(probs_for_correct_classes_expr))))

  return logloss

def App_test_error(sample_test, counts, sample_names, number_of_snp, table):
    """
  Вычисляет ошибку для каждого SNP

  Args:
     sample_test - matrix table по типу vcf.gz только генотипы 0, 1, 2 - получен на 5м шаге
     counts - hail table с частотами каждого SNP в популяциях
     sample_names - имя образцов для обучающей выборки
     number_of_snp - для какого SNP считаем вероятности
     table - hail table = annotationTraining.txt

  Returns:
    Значение logloss error для каждого SNP в файле.
  """
    #Считаем матрицу вероятностей
    res = prediction_naive_bayes(sample_test, counts, sample_names, number_of_snp)
    # Преобразуем numpy массив в pandas DataFrame
    df = pd.DataFrame(res, columns=pops)
    #Добавляем колонку с именами образцов
    df["sample"] = sample_names
    #Добавляем колонку с именами популяций
    population = table.super_pop.collect()
    df["super_pop"] = population
    ht = hl.Table.from_pandas(df)
    error = logloss_error(ht)
    return error

2024-06-26 05:47:42.930 Hail: WARN: Name collision: field 'sample' already in object dict. 
  This field must be referenced with __getitem__ syntax: obj['sample']
2024-06-26 05:47:42.941 Hail: INFO: Reading table without type imputation
  Loading field 'sample' as type str (not specified)
  Loading field 'pop' as type str (not specified)
  Loading field 'super_pop' as type str (not specified)
  Loading field 'gender' as type str (not specified)


Сам 6ой шаг


In [ ]:
a = hl.import_table("annotationTraining.txt").select("sample","super_pop").key_by("sample")
#Создаем список с именами все анализируемых vcf файлов - шаг 5 - filenames
#Вводим глобальные переменнные
diploid = True
sample_names = [(re.search(r"Struct\(sample='(.*?)'\)", str(i)).group(1)) for i in a.key_by().select("sample").collect()]
#Количесвто AIM, которое мы хотим получить в итоге
numAIM = 10
#Частота встречаемости популяций - словарь
samples_in_deme = a.aggregate(hl.agg.counter(a.super_pop))
#Создаем пустой список для хранения названий информативных маркеров
sampleAIMs = []
pops = list(a.aggregate(hl.agg.collect_as_set(a.super_pop)))
#Ищем AIMset
sampleAIMs = find_aims_hail(filenames, a, numAIM, sampleAIMs)